In [11]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas
from skmap.misc import find_files, GoogleSheet, ttprint
import warnings
import time
from scipy.special import expit, logit
import warnings
import joblib
import pickle
from pathlib import Path
from trees_rf import TreesRandomForestRegressor
from model_fit import read_features, cfi_calc, parameter_fine_tuning, calc_ccc, separate_data, rscfi 
from model_fit import accuracy_plot, plot_top_features, pdp_hexbin, plot_histogram, calc_metrics, accuracy_strata_plot
import warnings
warnings.filterwarnings('ignore')
import os
import json
from datetime import datetime

data_path = '/home/xuemeng/work_xuemeng/soc/SoilHealthDataCube/soil_property_model_pipeline'
prop_list = ['soc','ocd', 'bulk.density.fe','ph.cacl2', 'ph.h2o','total.n', 'carbonates', 
             'texture1','texture2', 'extractable.p', 'extractable.k','cec'] #'coarse.mass', 'coarse.vol', 'bulk.density.tot', 'ec', 

space_list = ['log1p','log1p','normal','normal','normal','log1p','log1p','log1p','log1p','log1p','log1p','log1p'] 
version = '20250212'

# generate the production model with all the data we have

In [13]:
from sklearn.metrics import r2_score
dff = pd.read_parquet(f'{data_path}/soil_organized.pq')
dff['ocd_qa']=5
for ii in [1]:#range(0, len(prop_list)):
    # Define metadata ------------------------------------------------------------
    prop = prop_list[ii]
    ttprint(f'processing {prop}-----------------------------------')
    space = space_list[ii]
    output_folder = data_path + '/' + prop
    os.makedirs(output_folder, exist_ok=True)
    
    df = dff.loc[dff[prop].notna() & (dff[prop+'_qa']>2)]
    
    if space == 'log1p':
        tgt = f'{prop}_log1p'
        df[f'{prop}_log1p'] = np.log1p(df[prop])
    elif space == 'boxcox':
        tgt = f'{prop}_boxcox'
        cal[f'{prop}_boxcox'], fitted_lambda = boxcox(cal['ocd'], lmbda=None)
    else:
        tgt = prop
    
    covs = read_features(f'{output_folder}/feature_selected_{prop}_v{version}.txt')  # Read in
    
    model = joblib.load(f'{output_folder}/model_rf.{prop}_ccc.joblib')
    model.n_jobs = 90
    
    # Drop rows with NaN values in the target column
    
    # df = df.dropna(subset=[tgt])
    # df = df.reset_index(drop=True)
    df = df.dropna(subset=covs)
    y = df[tgt]
    X = df[covs]
    
#     nan_percentage = (X.isna().sum() / len(X)) * 100
#     nan_percentage = nan_percentage.sort_values(ascending=False)
#     print("\nNaN Percentage per Column (Sorted):")
#     for nn, (col, perc) in enumerate(nan_percentage.items()):
#         print(f"{col}: {perc:.2f}%")
#         if nn > 10:
#             break
    
#     print(f"Number of samples: {X.shape[0]}")
#     X_np = X.to_numpy()
#     print(f"Number of covariates: {len(covs)}")
#     print(f"Percentage of NaN in X: {np.sum(np.sum(np.isnan(X_np))) / X_np.size * 100:.2f}%, filling them with median")
   
#     X = X.fillna(X.median())
    model.fit(X, y)
    
    y_pred = model.predict(X)
    r2 = r2_score(y, y_pred)
    
    print(f"R² score on the training set: {r2:.4f}")
    joblib.dump(model, f'{data_path}/production/model_rf.{prop}_production_v{version}.joblib')

[10:43:51] processing ocd-----------------------------------
R² score on the training set: 0.8761


In [2]:
for ii in range(0, len(prop_list)):
    df = pd.read_parquet(f'{data_path}/material/soil_pnts_organized.pq')
    
    # Define metadata ------------------------------------------------------------
    prop = prop_list[ii]
    ttprint(f'processing {prop}-----------------------------------')
    space = space_list[ii]
    output_folder = data_path + '/' + prop
    os.makedirs(output_folder, exist_ok=True)
    
    if space == 'log1p':
        df[f'{prop}_log1p'] = np.log1p(df[prop])
        tgt = f'{prop}_log1p'
    elif space == 'boxcox':
        tgt = f'{prop}_boxcox'
        cal[f'{prop}_boxcox'], fitted_lambda = boxcox(cal['ocd'], lmbda=None)
    else:
        tgt = prop
    
    covs = read_features(f'{output_folder}/feature_selected_{prop}_v{version}.txt')  # Read in
    
    model = joblib.load(f'{output_folder}/model_rf.{prop}_ccc.joblib')
    model.n_jobs = 90
    
    # Drop rows with NaN values in the target column
    df = df.dropna(subset=[tgt])
    df = df.reset_index(drop=True)
    y = df[tgt]
    X = df[covs]
    
    
    nan_percentage = (X.isna().sum() / len(X)) * 100
    nan_percentage = nan_percentage.sort_values(ascending=False)
    print("\nNaN Percentage per Column (Sorted):")
    for nn, (col, perc) in enumerate(nan_percentage.items()):
        print(f"{col}: {perc:.2f}%")
        if nn > 10:
            break
    
    print(f"Number of samples: {X.shape[0]}")
    X_np = X.to_numpy()
    print(f"Number of covariates: {len(covs)}")
    print(f"Percentage of NaN in X: {np.sum(np.sum(np.isnan(X_np))) / X_np.size * 100:.2f}%, filling them with median")
   
    X = X.fillna(X.median())
    model.fit(X, y)
    
    y_pred = model.predict(X)
    r2 = r2_score(y, y_pred)
    
    print(f"R² score on the training set: {r2:.4f}")
    
    # joblib.dump(model, f'{data_path}/production/model_rf.{prop}_production_v{version}.joblib')

[16:52:44] processing ocd-----------------------------------

NaN Percentage per Column (Sorted):
green_glad.swa.ard2_m_30m_s_YYYY0501_YYYY0630_go_epsg.4326_v1: 1.70%
ndvi_glad.swa.ard2_m_30m_s_YYYY0701_YYYY0831_go_epsg.4326_v1: 1.70%
blue_glad.swa.ard2_m_30m_s_YYYY0501_YYYY0630_go_epsg.4326_v1: 1.70%
blue_glad.swa.ard2_p25_30m_s_YYYY0101_YYYY1231_go_epsg.4326_v1: 1.70%
red_glad.swa.ard2_m_30m_s_YYYY0501_YYYY0630_go_epsg.4326_v1: 1.70%
ndsi_glad.swa.ard2_m_30m_s_YYYY0701_YYYY0831_go_epsg.4326_v1: 1.70%
ndsi_glad.swa.ard2_p25_30m_s_YYYY0101_YYYY1231_go_epsg.4326_v1: 1.70%
red_glad.swa.ard2_p25_30m_s_YYYY0101_YYYY1231_go_epsg.4326_v1: 1.70%
ndwi_glad.swa.ard2_m_30m_s_YYYY0701_YYYY0831_go_epsg.4326_v1: 1.70%
ndvi_glad.swa.ard2_m_30m_s_YYYY0501_YYYY0630_go_epsg.4326_v1: 1.70%
savi_glad.swa.ard2_m_30m_s_YYYY0701_YYYY0831_go_epsg.4326_v1: 1.70%
ndvi_glad.swa.ard2_p25_30m_s_YYYY0101_YYYY1231_go_epsg.4326_v1: 1.70%
Number of samples: 215891
Number of covariates: 60
Percentage of NaN in X: 0.65